### Seção 1: Importações de Bibliotecas

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr, when, to_timestamp, year, month, dayofmonth, hour
from pyspark.sql.types import IntegerType, StringType, TimestampType
import os
import time
import psycopg2

### Seção 2: Inicialização da Sessão Spark

In [ ]:
print("Iniciando a sessão Spark...")
spark = SparkSession.builder \
    .appName("Acidentes_Aereos_ETL_Raw_to_Silver") \
    .getOrCreate()
print("Sessão Spark iniciada com sucesso!")

### Seção 3: Definição de Caminhos

In [ ]:
raw_path = "../Data_Layer/raw"
print(f"Lendo arquivos da origem (RAW) de: {raw_path}")

print("Lendo arquivo da camada RAW...")
try:
    df_raw = spark.read.option("header", "true").option("sep", ";").option("encoding", "UTF-8").csv(os.path.join(raw_path, "data_raw.csv"))
    print("Arquivo RAW carregado com sucesso!")
    print(f"Total de registros: {df_raw.count()}")
except Exception as e:
    print(f"Erro ao ler o arquivo CSV. Verifique o caminho e a permissão dos arquivos. Erro: {e}")
    spark.stop()
    raise e

### Seção 4: Leitura dos Dados Brutos (RAW)

In [ ]:
print("Visualizando estrutura dos dados RAW...")
df_raw.printSchema()
print("\nPrimeiras 5 linhas:")
df_raw.show(5, truncate=False)

### Seção 5: Transformação e Unificação dos Dados

In [ ]:
print("Iniciando a transformação e limpeza dos dados...")

# 1. Remover espaços em branco das colunas
print("1️⃣  Removendo espaços em branco dos nomes das colunas...")
for old_col in df_raw.columns:
    new_col = old_col.strip()
    df_raw = df_raw.withColumnRenamed(old_col, new_col)
print("   ✅ Colunas padronizadas")

# 2. Converter coluna de data e hora
print("\n2️⃣  Convertendo data e hora da ocorrência...")
df_silver = df_raw.withColumn(
    "Data e Hora da Ocorrencia",
    to_timestamp(col("Data e Hora da Ocorrencia"), "dd/MM/yyyy HH:mm")
)
print("   ✅ Data convertida para TimestampType")

# 3. Extrair componentes de data
print("\n3️⃣  Extraindo componentes de data (Ano, Mês, Dia, Hora)...")
df_silver = df_silver.withColumn("Ano", year(col("Data e Hora da Ocorrencia")))
df_silver = df_silver.withColumn("Mes", month(col("Data e Hora da Ocorrencia")))
df_silver = df_silver.withColumn("Dia", dayofmonth(col("Data e Hora da Ocorrencia")))
df_silver = df_silver.withColumn("Hora", hour(col("Data e Hora da Ocorrencia")))
print("   ✅ Componentes de data extraídos")

# 4. Converter colunas numéricas
print("\n4️⃣  Convertendo colunas numéricas...")
numeric_cols = [
    "Total de Fatalidades no Acidente",
    "Total de Recomendacoes",
    "Total de Aeronaves Envolvidas",
    "Quantidade de Assentos na Aeronave",
    "Ano de Fabricacao da Aeronave"
]

for col_name in numeric_cols:
    if col_name in df_silver.columns:
        df_silver = df_silver.withColumn(col_name, col(col_name).cast(IntegerType()))

print(f"   ✅ {len([c for c in numeric_cols if c in df_silver.columns])} colunas convertidas para IntegerType")

# 5. Preencher valores nulos com 0 nas colunas numéricas
print("\n5️⃣  Preenchendo valores nulos com 0...")
for col_name in numeric_cols:
    if col_name in df_silver.columns:
        df_silver = df_silver.fillna({col_name: 0})
        
print("   ✅ Valores nulos preenchidos")

# 6. Criar indicador de severidade
print("\n6️⃣  Criando indicador de severidade...")
df_silver = df_silver.withColumn(
    "Nivel_Severidade",
    when(col("Total de Fatalidades no Acidente") > 50, "CRÍTICO")
    .when(col("Total de Fatalidades no Acidente") > 10, "GRAVE")
    .when(col("Total de Fatalidades no Acidente") > 0, "MODERADO")
    .otherwise("LEVE")
)
print("   ✅ Indicador 'Nivel_Severidade' criado")

# 7. Filtrar registros com data válida (opcional)
print("\n7️⃣  Filtrando registros com data válida...")
df_silver = df_silver.filter(col("Data e Hora da Ocorrencia").isNotNull())
print(f"   ✅ Registros após filtragem: {df_silver.count()}")

print("\n✅ Transformação de dados concluída!")
df_silver.printSchema()

### Seção 6: Carga dos Dados no Banco de Dados PostgreSQL

In [ ]:
print("\nIniciando a carga de dados no banco de dados...")
jdbc_hostname = os.getenv("DB_HOST", "localhost")
jdbc_port     = os.getenv("DB_PORT", "5432")
jdbc_database = os.getenv("DB_NAME", "acidentes_aereos")
db_user       = os.getenv("DB_USER", "postgres")
db_password   = os.getenv("DB_PASSWORD", "postgres")
table_name    = "public.acidentes_silver"
jdbc_url = f"jdbc:postgresql://{jdbc_hostname}:{jdbc_port}/{jdbc_database}"
connection_properties = {"user": db_user, "password": db_password, "driver": "org.postgresql.Driver"}

retries = 10
wait_seconds = 5
for i in range(retries):
    try:
        print("Tentando conectar ao banco de dados...")
        conn = psycopg2.connect(host=jdbc_hostname, dbname=jdbc_database, user=db_user, password=db_password, port=jdbc_port)
        conn.close()
        print("✅ Conexão com o banco de dados bem-sucedida!")
        break
    except psycopg2.OperationalError as e:
        print(f"⏳ Banco de dados não está pronto. Tentando novamente em {wait_seconds} segundos...")
        time.sleep(wait_seconds)
        if i == retries - 1:
            print("❌ Não foi possível conectar ao banco de dados. Abortando.")
            spark.stop()
            raise e

try:
    print(f"Iniciando a escrita de dados na tabela: {table_name}")
    
    df_silver.write \
        .mode("overwrite") \
        .jdbc(url=jdbc_url, table=table_name, properties=connection_properties)
        
    print(f"✅ Tabela '{table_name}' populada com sucesso no banco de dados '{jdbc_database}'!")

except Exception as e:
    print(f"❌ Ocorreu um erro ao salvar no banco de dados: {e}")
    spark.stop()
    raise e

### Seção 7: Validação e Finalização

In [ ]:
print("\nValidando a tabela SILVER (amostra):")
df_silver.orderBy(col("Ano").desc()).show(10, truncate=False)

print("\n=== ESTATÍSTICAS FINAIS ===")
print(f"Total de registros processados: {df_silver.count()}")
print(f"Colunas na tabela SILVER: {len(df_silver.columns)}")

print("\nDistribuição por Nível de Severidade:")
df_silver.groupBy("Nivel_Severidade").count().orderBy(col("count").desc()).show()

print("\n🚀 Job ETL (Raw → Silver → Database) finalizado com sucesso!")
spark.stop()